In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap
import lime
import lime.lime_tabular
from datetime import datetime
from pathlib import Path
from IPython.display import display
import warnings

warnings.filterwarnings('ignore')

# Enhanced Academic publication style settings
plt.style.use('seaborn-v0_8-paper')
sns.set_context("paper", font_scale=1.6)
sns.set_palette("deep")

# Configure matplotlib for publication quality with BOLD and BIGGER fonts
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']
plt.rcParams['font.size'] = 13
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'black'
plt.rcParams['legend.title_fontsize'] = 13
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['xtick.major.width'] = 1.2
plt.rcParams['ytick.major.width'] = 1.2

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("Libraries loaded successfully!")
print("Enhanced academic publication style configured")
print("Experiment 2: WITH Artist Features (414 features)")

In [ ]:
# Set paths for Experiment 2
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
FEATURES_DIR = PROJECT_ROOT / "features"
MODELS_DIR = PROJECT_ROOT / "models" / "saved" / "experiment2_with_artist"
RESULTS_DIR = PROJECT_ROOT / "results" / "metrics" / "experiment2_with_artist"
FIGURES_DIR = PROJECT_ROOT / "results" / "figures"

# Create directories
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Configuration - Top models per target for Experiment 2
TOP_MODELS = {
    'valence': 'XGBoost_tuned',
    'energy': 'XGBoost_tuned',
    'danceability': 'CatBoost',
    'popularity': 'CatBoost_tuned'
}

TARGETS = list(TOP_MODELS.keys())

print(f"Project root: {PROJECT_ROOT}")
print(f"Experiment 2: WITH Artist Features")
print(f"Targets and their top models:")
for target, model in TOP_MODELS.items():
    print(f"   - {target:12s}: {model}")

---
## 📋 Load Feature Names

In [ ]:
def load_feature_names():
    """Load feature names from metadata with proper audio feature names"""
    feature_names = []
    
    # Audio features (23 for Experiment 2 - includes 2 artist features)
    audio_names_path = FEATURES_DIR / "audio_feature_names.txt"
    if audio_names_path.exists():
        with open(audio_names_path, 'r') as f:
            audio_features = [line.strip() for line in f 
                            if line.strip() and not line.strip().startswith('#')]
        feature_names.extend(audio_features)
        print(f"Loaded {len(audio_features)} audio features from file")
    else:
        # Fallback: use standard Spotify audio feature names
        audio_features = [
            'danceability', 'energy', 'loudness', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'tempo',
            'genre_Rock', 'genre_Pop', 'genre_Hip-Hop', 'genre_Electronic',
            'genre_Jazz', 'genre_Classical', 'genre_Country', 'genre_R&B',
            'genre_Indie', 'genre_Metal', 'year_normalized', 'mode',
            'key_sin', 'log_total_artist_followers', 'avg_artist_popularity'
        ]
        feature_names.extend(audio_features)
        print(f"Audio feature file not found, using default names")
    
    # Text stats (5)
    text_stats = ['word_count', 'unique_word_count', 'unique_ratio', 'avg_word_length', 'char_count']
    feature_names.extend(text_stats)
    
    # Sentiment (2)
    sentiment = ['sentiment_polarity', 'sentiment_subjectivity']
    feature_names.extend(sentiment)
    
    # Embeddings (384)
    embedding_names = [f'emb_{i}' for i in range(384)]
    feature_names.extend(embedding_names)
    
    return feature_names

feature_names = load_feature_names()
print(f"Total features loaded: {len(feature_names)}")
print(f"\nFirst 32 features (including artist features):")
for i, name in enumerate(feature_names[:32]):
    print(f" {i:3d}. {name}")

In [ ]:
def load_validation_data():
    """Load validation features and targets from .npy files (Experiment 2)"""
    try:
        # Load feature components (23 audio features including artist)
        X_audio = np.load(FEATURES_DIR / "X_val_audio.npy")
        X_text_stats = np.load(FEATURES_DIR / "X_val_text_stats.npy")
        X_sentiment = np.load(FEATURES_DIR / "X_val_sentiment.npy")
        X_embeddings = np.load(FEATURES_DIR / "X_val_embeddings.npy")
        
        # Concatenate features (23 + 5 + 2 + 384 = 414)
        X_val_arr = np.hstack([X_audio, X_text_stats, X_sentiment, X_embeddings])
        
        # Create DataFrame
        X_val = pd.DataFrame(X_val_arr, columns=feature_names)
        
        # Load targets
        y_val = pd.DataFrame({
            'valence': np.load(FEATURES_DIR / "y_val_valence.npy"),
            'energy': np.load(FEATURES_DIR / "y_val_energy.npy"),
            'danceability': np.load(FEATURES_DIR / "y_val_danceability.npy"),
            'popularity': np.load(FEATURES_DIR / "y_val_popularity.npy")
        })
        
        return X_val, y_val
    except Exception as e:
        print(f"Error loading validation data: {e}")
        return None, None

X_val, y_val = load_validation_data()
if X_val is not None:
    print(f"Loaded Experiment 2 validation data: {X_val.shape}")
    print(f"Targets shape: {y_val.shape}")
    print(f"Features: 23 audio (incl. artist) + 5 text + 2 sentiment + 384 embeddings = 414 total")
else:
    print("Failed to load validation data")

---
## 🔧 Define Helper Functions

In [ ]:
def get_feature_importance(model, model_name, feature_names):
    """Extract feature importance from a model"""
    
    importance = None
    method = None
    
    # Tree-based models with feature_importances_
    if hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_
        method = 'feature_importances_'
    
    # MLPRegressor - use first layer weights
    elif hasattr(model, 'coefs_') and len(model.coefs_) > 0:
        # Sum absolute weights from input to first hidden layer
        first_layer_weights = model.coefs_[0]
        importance = np.abs(first_layer_weights).sum(axis=1)
        method = 'mlp_first_layer'
    
    # Linear models with coef_
    elif hasattr(model, 'coef_'):
        importance = np.abs(model.coef_)
        if importance.ndim > 1:
            importance = importance.mean(axis=0)
        method = 'coef_'
    
    if importance is not None:
        # Verify shape
        if len(importance) != len(feature_names):
            print(f"Shape mismatch: {len(importance)} vs {len(feature_names)}")
            return None
        
        # Normalize
        if importance.sum() > 0:
            importance = importance / importance.sum()
        
        return pd.DataFrame({
            'feature': feature_names,
            'importance': importance,
            'method': method
        }).sort_values('importance', ascending=False)
    
    return None

def analyze_model_importance(target, model_name, feature_names):
    """Analyze feature importance for a specific model and target"""
    
    model_path = MODELS_DIR / f"{model_name}_{target}.pkl"
    
    if not model_path.exists():
        return None
    
    try:
        model = joblib.load(model_path)
        importance_df = get_feature_importance(model, model_name, feature_names)
        
        if importance_df is not None:
            importance_df['target'] = target
            importance_df['model'] = model_name
            return importance_df
        return None
            
    except Exception as e:
        print(f"Error loading {model_name}: {e}")
        return None

print("Helper functions defined")

---
## 🚀 Extract Feature Importance from All Models

In [ ]:
print("Extracting feature importance for top models...\n")

all_importance = []

for target, model_name in TOP_MODELS.items():
    print(f"{target.upper()} (Model: {model_name})")
    
    result = analyze_model_importance(target, model_name, feature_names)
    if result is not None:
        all_importance.append(result)
        top_feat = result.iloc[0]['feature']
        top_imp = result.iloc[0]['importance']
        print(f"   Success → Top feature: {top_feat} ({top_imp:.3f})")
    else:
        print(f"   Failed to load or analyze")

# Combine all results
if all_importance:
    all_importance_df = pd.concat(all_importance, ignore_index=True)
    print(f"\nExtracted importance for {len(all_importance)} top models")
else:
    print("\nNo importance data extracted")

In [ ]:
# Save raw data
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
raw_path = RESULTS_DIR / f"feature_importance_raw_{timestamp}.csv"

print(f"Saving to: {raw_path}")
print(f"DataFrame shape: {all_importance_df.shape}")
print(f"Directory exists: {RESULTS_DIR.exists()}")

all_importance_df.to_csv(raw_path, index=False)

# Verify file was saved
if raw_path.exists():
    file_size = raw_path.stat().st_size / 1024
    print(f"Saved raw importance data: {raw_path.name} ({file_size:.1f} KB)")
else:
    print(f"ERROR: File was not saved!")

---
## 📊 Top 20 Features per Target

In [ ]:
print("TOP 20 FEATURES PER TARGET (averaged across models)\n")

summary_data = []

for target in TARGETS:
    target_df = all_importance_df[all_importance_df['target'] == target]
    top20 = target_df.groupby('feature')['importance'].mean().nlargest(20)
    
    print(f"\n{'='*70}")
    print(f"{target.upper()}")
    print(f"{'='*70}")
    
    for i, (feat, imp) in enumerate(top20.items(), 1):
        print(f"   {i:2d}. {feat:<30s} {imp:.4f}")
        summary_data.append({
            'target': target,
            'rank': i,
            'feature': feat,
            'avg_importance': imp
        })

summary_df = pd.DataFrame(summary_data)
summary_path = RESULTS_DIR / f"feature_importance_summary_{timestamp}.csv"
summary_df.to_csv(summary_path, index=False)
print(f"\nSaved summary: {summary_path.name}")

---
## 📈 Visualization 1: Top 20 Features per Target

In [ ]:
def plot_top_features(all_importance, target, top_n=20):
    """Plot top N features for a target with enhanced academic styling"""
    target_df = all_importance[all_importance['target'] == target].copy()
    agg_importance = target_df.groupby('feature')['importance'].mean().sort_values(ascending=False)
    top_features = agg_importance.head(top_n)
    
    fig, ax = plt.subplots(figsize=(14, 9))
    colors = plt.cm.viridis(np.linspace(0, 1, len(top_features)))
    
    bars = ax.barh(range(len(top_features)), top_features.values, 
                   color=colors, edgecolor='black', alpha=0.85, linewidth=1.5)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features.index, fontweight='bold', fontsize=12)
    ax.invert_yaxis()
    ax.set_xlabel('Average Importance', fontsize=15, fontweight='bold', labelpad=10)
    ax.set_title(f'Experiment 2: Top {top_n} Features for {target.upper()}', 
                 fontsize=17, fontweight='bold', pad=20)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2)
    
    # Add value labels
    for bar, val in zip(bars, top_features.values):
        ax.text(val + 0.001, bar.get_y() + bar.get_height()/2, 
                f'{val:.3f}', va='center', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    return fig

# Generate plots for all targets
for target in TARGETS:
    fig = plot_top_features(all_importance_df, target, top_n=20)
    fig_path = FIGURES_DIR / f"experiment2_shap_feature_importance_{target}.png"
    fig.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Saved: {fig_path.name}")

---
## 🧠 Explainable AI (XAI): SHAP Global Analysis
SHAP (SHapley Additive exPlanations) provides a mathematically grounded way to explain model predictions. Unlike simple feature importance, SHAP shows the **direction** of the effect (e.g., does high energy increase or decrease valence?).

In [ ]:
def run_shap_analysis(target, model_name, X_sample):
    """Run SHAP analysis for a specific model and target with enhanced styling"""
    model_path = MODELS_DIR / f"{model_name}_{target}.pkl"
    
    if not model_path.exists():
        print(f"Model not found: {model_path}")
        return
    
    print(f"Running SHAP for {target} ({model_name})...")
    model = joblib.load(model_path)
    
    # Create explainer
    # For XGBoost and CatBoost, we can use TreeExplainer which is very fast
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)
    
    # Plot Summary with enhanced styling
    plt.figure(figsize=(14, 10))
    shap.summary_plot(shap_values, X_sample, show=False, plot_size=(14, 10), max_display=20)
    plt.title(f"SHAP - {target.upper()} ({model_name})", 
              fontsize=17, fontweight='bold', pad=20)
    # plt.xlabel('SHAP value (impact on model output)', fontsize=15, fontweight='bold')
    plt.tick_params(axis='both', labelsize=12, width=1.2)
    
    fig_path = FIGURES_DIR / f"experiment2_shap_summary_{target}.png"
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Saved SHAP summary: {fig_path.name}")
    
    return shap_values

# Use a sample of 500 instances for SHAP to keep it fast
SHAP_SAMPLE_SIZE = 500
X_shap_sample = X_val.sample(min(SHAP_SAMPLE_SIZE, len(X_val)), random_state=42)

shap_results = {}
for target, model_name in TOP_MODELS.items():
    shap_val = run_shap_analysis(target, model_name, X_shap_sample)
    shap_results[target] = shap_val

---
## 🔍 Explainable AI (XAI): LIME Local Analysis
LIME (Local Interpretable Model-agnostic Explanations) helps us understand **why** the model made a specific prediction for a single song. It creates a local linear model around that specific instance.

In [ ]:
def run_lime_analysis(target, model_name, X_train_sample, X_test_instance, instance_idx):
    """Run LIME for a specific instance with enhanced styling"""
    model_path = MODELS_DIR / f"{model_name}_{target}.pkl"
    model = joblib.load(model_path)
    
    # Create LIME explainer
    explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=X_train_sample.values,
        feature_names=X_train_sample.columns.tolist(),
        class_names=[target],
        mode='regression'
    )
    
    # Explain prediction
    exp = explainer.explain_instance(
        X_test_instance.values, 
        model.predict, 
        num_features=10
    )
    
    # Plot with enhanced styling
    fig = exp.as_pyplot_figure()
    fig.set_size_inches(14, 7)
    plt.title(f"Experiment 2: LIME Explanation - {target.upper()} (Instance {instance_idx})", 
              fontsize=17, fontweight='bold', pad=20)
    plt.xlabel('Feature Contribution', fontsize=15, fontweight='bold')
    plt.tick_params(axis='both', labelsize=12, width=1.2)
    plt.tight_layout()
    
    fig_path = FIGURES_DIR / f"experiment2_lime_{target}_instance_{instance_idx}.png"
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    return exp

# Pick an instance to explain (e.g., the first one in our validation set)
instance_to_explain = X_val.iloc[0]
print(f"Explaining prediction for first instance in validation set...")

for target, model_name in TOP_MODELS.items():
    print(f"\nLIME for {target}...")
    run_lime_analysis(target, model_name, X_shap_sample, instance_to_explain, 0)

---
## 📊 Visualization 2: Feature Group Importance

In [ ]:
def create_feature_groups():
    """Define feature groups including artist features for Experiment 2"""
    groups = {
        'audio_continuous': list(range(0, 7)),   # acousticness, instrumentalness, speechiness, liveness, loudness, tempo, duration_ms
        'year': [7],
        'mode': [8],
        'key_cyclical': [9, 10],  # key_sin, key_cos
        'genre': list(range(11, 21)),  # 10 genre features (Blues, Classical, Country, Electronic, Folk, Hip-Hop, Jazz, Pop, R&B, Rock)
        'artist': [21, 22],  # log_total_artist_followers + avg_artist_popularity
        'text_stats': list(range(23, 28)),  # word_count, unique_word_count, unique_ratio, avg_word_length, char_count
        'sentiment': list(range(28, 30)),  # sentiment_polarity, sentiment_subjectivity
        'embeddings': list(range(30, 414)),  # 384 embedding features
    }
    return groups

def aggregate_by_group(importance_df, feature_names):
    """Aggregate feature importance by group"""
    groups = create_feature_groups()
    group_importance = {}
    
    for group_name, indices in groups.items():
        group_features = [feature_names[i] for i in indices if i < len(feature_names)]
        mask = importance_df['feature'].isin(group_features)
        group_importance[group_name] = importance_df.loc[mask, 'importance'].sum()
    
    return pd.Series(group_importance).sort_values(ascending=False)

# Plot for Top Models with enhanced styling
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#28A745', '#6C757D', '#17A2B8', '#FFC107', '#FF69B4']

for ax, target in zip(axes.flatten(), TARGETS):
    model_name = TOP_MODELS[target]
    target_df = all_importance_df[(all_importance_df['target'] == target) & (all_importance_df['model'] == model_name)]
    
    if target_df.empty:
        ax.text(0.5, 0.5, f"No data for {target}", ha='center', fontsize=14, fontweight='bold')
        continue
        
    group_imp = aggregate_by_group(target_df, feature_names)
    avg_group = group_imp.sort_values(ascending=True)
    
    bars = ax.barh(range(len(avg_group)), avg_group.values, 
                  color=colors[:len(avg_group)], edgecolor='black', alpha=0.85, linewidth=1.5)
    ax.set_yticks(range(len(avg_group)))
    ax.set_yticklabels(avg_group.index, fontweight='bold', fontsize=12)
    ax.set_xlabel('Importance', fontsize=14, fontweight='bold', labelpad=8)
    ax.set_title(f'{target.upper()} ({model_name})', fontsize=15, fontweight='bold', pad=15)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2)
    
    # Add value labels
    for bar, val in zip(bars, avg_group.values):
        ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, 
                f'{val:.2%}', va='center', fontsize=11, fontweight='bold')

plt.suptitle('Experiment 2: Feature Group Importance for Top Models (With Artist Features)', 
             fontsize=17, fontweight='bold', y=0.995)
plt.tight_layout()
fig_path = FIGURES_DIR / "experiment2_feature_group_importance_top_models.png"
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {fig_path.name}")

---
## 🔑 Key Insights: Embedding Contribution

In [ ]:
print("EMBEDDING CONTRIBUTION ANALYSIS\n")
print("="*70)

embedding_cols = [f'emb_{i}' for i in range(384)]

insights = []
for target in TARGETS:
    target_df = all_importance_df[all_importance_df['target'] == target]
    emb_importance = target_df[target_df['feature'].isin(embedding_cols)]['importance'].sum()
    total_importance = target_df['importance'].sum()
    emb_pct = (emb_importance / total_importance * 100) if total_importance > 0 else 0
    
    print(f"   {target.upper():15s} → Embeddings: {emb_pct:5.1f}% of total importance")
    insights.append({'target': target, 'embedding_contribution_pct': emb_pct})

insights_df = pd.DataFrame(insights)
display(insights_df.style.format({'embedding_contribution_pct': '{:.2f}%'}).background_gradient(
    subset=['embedding_contribution_pct'], cmap='YlOrRd'
))